### 1. Data Pre-processing

We will only be doing weekly forecasts for item 1.

Importing the dataset:

In [118]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"D:\ML\Datasets\Demand Forecasting\train.csv")
df = df[df['item'] == 1]
df.head(5)

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


Group sales across all stores into weekly periods:

In [119]:
df['date'] = pd.to_datetime(df['date']) - pd.to_timedelta(7, unit='d')
df = df.filter(['date', 'sales']).groupby([pd.Grouper(key='date', freq='W-MON')]).sum().reset_index()
df.head(5)

,date,sales
0,2012-12-31,894
1,2013-01-07,863
2,2013-01-14,867
3,2013-01-21,816
4,2013-01-28,969


In [120]:
df['shift_sale'] = df['sales'].shift(1)
df = df.iloc[1:]
df.head(5)

,date,sales,shift_sale
1,2013-01-07,863,894.0
2,2013-01-14,867,863.0
3,2013-01-21,816,867.0
4,2013-01-28,969,816.0
5,2013-02-04,920,969.0


Algorithm nursery:

In [121]:
for i in range(3, -1, -1):
    #print(i)
    for j in range(i):
        print(j)

0
1
2
0
1
0


In [122]:
sales = [1, 3, 3, 5, 1, 2, 3, 3]

In [123]:
sum = 0
week_avg = []
for row in range(len(sales) - 4):
        for row in range(row, row + 4):
            sum += sales[row]
        week_avg.append(sum / 4)
        sum = 0
week_avg   

[3.0, 3.0, 2.75, 2.75]

In [124]:
for i in range(4):
    print(i)

0
1
2
3


Define a function to calculate rolling four-week average:

In [125]:
def four_week_avg(sales):
    sum = 0
    week_avg = []
    for i in range(3, -1, -1):
        for j in range(i):
            sum += sales[j]
        if(i!=0):
            week_avg.append(sum / i)
        sum = 0
    week_avg.append(sales[0])
    week_avg.reverse()
    #return week_avg
    for row in range(len(sales) - 4):
        for row in range(row, row + 4):
            sum += sales[row]
        week_avg.append(sum / 4)
        sum = 0
    return week_avg    

In [126]:
df['week_avg'] = four_week_avg(df['sales'].tolist())
df.head()

,date,sales,shift_sale,week_avg
1,2013-01-07,863,894.0,863.000000
2,2013-01-14,867,863.0,863.000000
3,2013-01-21,816,867.0,865.000000
4,2013-01-28,969,816.0,848.666667
5,2013-02-04,920,969.0,878.750000


### 2. Model and Evaluation

In [127]:
from matplotlib import pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, explained_variance_score, max_error

In [128]:
test = df.iloc[-52:] # one year worth of sales
df = df.iloc[:-52]

In [130]:
X = df.drop(['sales', 'date'], axis=1)
y = df['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = svm.SVR(C=1, kernel='linear', degree=8, gamma='scale', coef0=10)
clf.fit(X_train, y_train)

SVR(C=1, coef0=10, degree=8, kernel='linear')

In [132]:
predictions = clf.predict(X_test)
print()